```{contents}
```
## MCP (Model Context Protocol) Integration


**MCP (Model Context Protocol)** is an open protocol that allows LLM applications to **connect to external tools, data sources, and services** in a standardized way.

It turns external systems into **plug-and-play tools** for AI models.

Instead of hard-coding integrations:

```
LLM → Custom API Logic → External Tool
```

With MCP:

```
LLM → MCP Client → MCP Server → External Tool
```

---

### Why MCP Matters

| Problem              | MCP Solution        |
| -------------------- | ------------------- |
| Tool sprawl          | Standard interface  |
| Custom glue code     | Reusable connectors |
| Tight coupling       | Loose coupling      |
| Scaling integrations | Tool ecosystem      |

---

### Where MCP Fits in the Architecture

```
Client App / Agent
      ↓
MCP Client Library
      ↓
MCP Server (tool gateway)
      ↓
Databases / APIs / Services
```

---

### MCP Core Components

| Component  | Role                         |
| ---------- | ---------------------------- |
| MCP Server | Exposes tools                |
| MCP Client | Consumes tools               |
| Tools      | Functions the model can call |
| Resources  | External data                |
| Prompts    | Reusable prompt templates    |

---

### Building a Simple MCP Server

#### Demonstration

```python
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("math-server")

@mcp.tool()
def add(a: int, b: int) -> int:
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    return a * b

if __name__ == "__main__":
    mcp.run()
```

This exposes two tools: `add` and `multiply`.

---

### Connecting an MCP Client

#### Demonstration

```python
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import asyncio

async def main():
    params = StdioServerParameters(command="python", args=["math_server.py"])

    async with stdio_client(params) as (reader, writer):
        async with ClientSession(reader, writer) as session:
            tools = await session.list_tools()
            print(tools)

            result = await session.call_tool("add", {"a": 3, "b": 4})
            print(result)

asyncio.run(main())
```

---

### Integrating MCP with an LLM Agent

#### Demonstration (LangChain-style agent)

```python
from langchain.agents import initialize_agent, AgentType
from langchain.tools.mcp import MCPToolkit

toolkit = MCPToolkit.from_server(
    command="python",
    args=["math_server.py"]
)

agent = initialize_agent(
    toolkit.get_tools(),
    llm,
    agent=AgentType.OPENAI_FUNCTIONS
)

response = agent.run("What is 6 times 7?")
print(response)
```

The LLM automatically selects and calls the correct MCP tool.

---

### Adding External Services via MCP

Example: database query tool

```python
@mcp.tool()
def get_user(user_id: int) -> dict:
    return db.fetch_user(user_id)
```

The model can now reason about users without knowing DB code.

---

### Scaling MCP

```
Multiple MCP Servers → Registry → LLM Agents → Applications
```

Each server exposes specialized tool sets.

---

### Mental Model

```
MCP = USB port for AI systems
```

---

### Key Takeaways

* MCP standardizes tool integration for LLMs
* Decouples models from business logic
* Enables large, reusable tool ecosystems
* Foundation for scalable agent systems
